In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.


In [2]:
model = load_model('emojinator.h5')
print(model)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [3]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [5]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [6]:
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

0
1
2
3
4
5
6
7
8
9
10
11
12
5 0.92763674
1 0.9953424
1 0.9905054
1 0.9993154
1 0.99989223
1 0.99916196
1 0.9647257
1 0.99906486
1 0.9997923
8 0.6568066
8 0.7582966
8 0.63055134
5 0.97360736
1 0.9997937
1 0.99997914
1 0.9990363
1 0.9999428
1 0.99806947
1 0.8172268
2 0.64339477
1 0.9997081
5 0.98966837
1 0.96219265
1 0.99603695
1 0.99893564
1 0.99856865
1 0.99996746
1 0.9985328
1 0.98025686
1 0.9399128
8 0.62040657
8 0.816773
11 0.99983907
11 0.9999653
11 0.9999918
11 0.99999726
11 0.99999845
11 0.99999595
11 0.99999464
11 0.9999964
11 0.99999917
11 0.99999595
11 0.99999106
11 0.9999845
11 0.99999464
11 0.99999285
11 0.9999893
11 0.9999778
11 0.9999794
11 0.99997723
11 0.9999765
11 0.9999808
11 0.99998987
11 0.99996614
11 0.99998724
11 0.99983656
11 0.99996567
11 0.91819507
1 0.47134024
1 0.57167023
1 0.68880767
1 0.6850133
1 0.8739105
1 0.82327706
1 0.88238376
1 0.8352064
1 0.895189
1 0.89275026
1 0.90848285
1 0.91563356
1 0.9440118
1 0.9323793
1 0.9962831
1 0.99997795
1 0.98964614
1 0

4 0.9998211
4 0.99937314
4 0.99992085
4 0.9990144
4 0.99994314
4 0.99993515
4 0.99953747
4 0.9995129
4 0.9995834
4 0.9999386
4 0.99999857
4 0.9986985
4 0.9987224
4 0.99811065
4 0.99853575
4 0.99932766
4 0.9995047
4 0.9995907
4 0.9992642
4 0.9995297
4 0.99915814
4 0.99750084
4 0.99143183
3 0.9997646
3 0.6632317
9 0.6389119
6 0.6099371
7 0.98648155
9 0.9268069
9 0.99998045
9 0.70416355
9 0.99988604
9 0.7529708
4 0.6127178
4 0.51297045
7 0.43478307
4 0.48090133
4 0.8039462
4 0.9453065
4 0.9212458
4 0.9030169
4 0.5135019
4 0.8516028
3 0.5780418
3 0.92403746
3 0.86672515
11 0.57844204
8 0.992814
9 0.9981824
9 0.9973889
9 0.5832367
11 0.995148
11 0.743806
9 0.74172276
9 0.9496195
5 0.96545315
5 0.9999299
10 0.9924378
5 0.9999113
5 0.9297527
1 0.903454
1 0.96530163
1 0.99333054
1 0.99990714
1 0.99995935
1 0.9999752
1 0.99998844
1 0.9999943
1 0.9999831
1 0.9999697
1 0.9999528
1 0.9998841
1 0.9999914
1 0.9999846
1 0.99995685
1 0.9999833
1 0.9998167
1 0.99996865
1 0.9998727
1 0.99983144
1 0.9987

5 0.9996933
5 0.8212034
11 0.99263763
11 0.96325666
11 0.9630234
11 0.8962029
11 0.9952887
11 0.99963844
11 0.99999714
11 0.999998
11 0.9997446
11 0.99910873
11 0.92610073
11 0.98091406
11 0.97823983
11 0.968814
11 0.99578756
11 0.99963653
11 0.9989385
11 0.99668866
11 0.99999225
11 0.9997271
1 0.9962709
1 0.7390457
1 0.8436742
1 0.725125
1 0.73005015
5 0.7476225
5 0.66972035
5 0.7241546
5 0.73202825
1 0.7300287
5 0.7826723
5 0.78873706
5 0.8593003
5 0.9125882
5 0.824751
5 0.8297173
1 0.8070771
5 0.89355946
5 0.8464201
5 0.88749766
5 0.8712967
1 0.99921703
5 0.9175019
5 0.843599
5 0.91866857
5 0.815661
5 0.88600767
5 0.849557
5 0.95596904
1 0.99338746
1 0.99188906
2 0.9998683
1 0.89235455
1 0.90995646
1 0.9599338
1 0.976722
1 0.9942806
1 0.9837556
1 0.9929478
1 0.99089736
1 0.99450994
1 0.99441445
1 0.9915945
1 0.9947712
1 0.9963875
1 0.9955383
1 0.99766326
1 0.99797815
1 0.99735045
1 0.99572635
1 0.99914896
5 0.80811566
5 0.99978334
5 0.99988496
5 0.99999726
5 0.99999976
5 0.99999976


1 0.8604262
1 0.56709844
8 0.9245421
8 0.96377146
8 0.85808563
8 0.95278776
8 0.96795493
8 0.9399819
8 0.7477925
8 0.9707835
8 0.9718027
8 0.9569976
8 0.7099677
8 0.5298731
3 0.9229857
3 0.99323785
3 0.99742645
3 0.9966
3 0.99649495
3 0.998847
3 0.9949437
3 0.9995023
3 0.9995486
3 0.9999161
3 0.9307082
3 0.9997905
3 0.9991841
3 0.9968256
3 0.9578397
3 0.9748093
3 0.81219447
3 0.7945679
4 0.8530063
3 0.77719176
4 0.8871163
3 0.9811366
4 0.58525115
4 0.9329901
3 0.77046293
4 0.60532826
4 0.88286185
3 0.504354
4 0.81626135
4 0.9596816
3 0.5516481
4 0.85000026
3 0.9965534
3 0.99830186
3 0.9490731
3 0.9519806
3 0.96398336
3 0.99856347
3 0.99605787
3 0.99930537
3 0.99985874
3 0.9985429
3 0.9997676
3 0.99856097
3 0.9995092
3 0.99967146
4 0.6666945
3 0.6649089
4 0.51713663
3 0.8399353
4 0.690093
4 0.59897083
4 0.9454224
7 0.7716074
4 0.6792868
7 0.85070854
4 0.6906648
3 0.55227435
4 0.7630663
4 0.5750571
4 0.7606788
3 0.6855187
3 0.99075085
3 0.99945515
3 0.9973194
3 0.99947554
3 0.99868685
3 

7 0.8622223
7 0.60674685
4 0.85421455
3 0.9146086
4 0.9836483
4 0.75232726
4 0.97676283
4 0.9860459
4 0.8076518
4 0.82103384
4 0.9588416
4 0.9117775
4 0.7188528
5 0.62230927
4 0.6268323
5 0.88545084
4 0.8190173
4 0.73272276
4 0.9690616
4 0.52084506
4 0.5398432
3 0.7416547
4 0.9497271
4 0.87775844
4 0.81984764
5 0.610926
4 0.66571116
4 0.84779
4 0.6048138
4 0.4806892
5 0.78331107
5 0.8602485
5 0.88620526
5 0.9027964
5 0.8012746
5 0.71459544
5 0.8960803
5 0.8995866
4 0.8567564
4 0.83972627
4 0.6568602
3 0.9701487
3 0.9081289
3 0.960958
3 0.988555
3 0.89527786
3 0.5070316
3 0.9294833
3 0.99837816
3 0.98789304
3 0.90956295
3 0.9987877
3 0.974089
3 0.9995283
3 0.99880826
3 0.9979248
3 0.99948823
3 0.99773455
3 0.99951446
3 0.99997175
3 0.9999045
3 0.99975485
3 0.99997926
3 0.9998122
3 0.99995124
3 0.9999398
3 0.9995728
3 0.99956554
3 0.9997204
3 0.9998167
3 0.99992406
3 0.9999007
3 0.99993813
3 0.9998981
3 0.99979633
3 0.9998667
3 0.9998418
3 0.9999368
3 0.9998272
3 0.9998474
3 0.999567
3 0

...

9 0.9999795
9 0.8070305
9 0.98454684
9 1.0
9 0.9999465
9 0.98372066
6 0.8942745
9 0.9998154
9 0.9065148
9 0.99992716
9 0.9999987
9 0.9999894
9 0.98415655
9 0.9999771
9 0.9915924
9 0.9999981
9 0.86746866
6 0.9298436
10 0.8643654
10 0.9109384
6 0.99896324
6 0.95247555
2 0.9521836
6 0.9308364
6 0.7688206
6 0.981219
4 0.6981824
6 0.9045887
6 0.819175
9 0.77353483
6 0.6984063
6 0.6984693
10 0.63574374
9 0.77592784
6 0.58966315
9 0.99252796
9 0.33600423
10 0.72236073
9 0.3381464
4 0.5411239
10 0.97204214
4 0.7262754
10 0.78475356
4 0.57975084
9 0.99976283
6 0.8889418
6 0.7354388
9 0.99350256
10 0.6658144
9 0.9884344
6 0.92775154
9 0.924546
10 0.8936775
6 0.5897889
6 0.34725133
4 0.33094624
6 0.9559188
10 0.6077063
4 0.78377163
10 0.36402747
4 0.42883852
9 0.37286058
9 0.9568123
4 0.53649443
6 0.556858
4 0.39244947
9 0.47981492
10 0.39285806
6 0.79634154
6 0.34617537
4 0.22055669
6 0.40550706
6 0.9683335
9 0.6984667
4 0.57216674
9 0.8192386
6 0.5106709
4 0.5238285
6 0.2791795
9 0.9148195
9 0.

9 0.99999976
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 0.9999999
9 0.9999995
9 0.99999964
9 1.0
9 0.999995
9 0.99999595
9 0.99999833
9 0.999997
9 0.9999993
9 0.999995
9 0.99994445
9 0.9860698
9 0.9999969
9 0.99996305
9 0.99989533
9 0.999622
9 0.9997695
9 0.9979254
9 0.99989235
9 0.9999583
9 0.9999428
9 0.9998945
9 0.9999436
9 0.9988979
9 0.9974246
9 0.9996432
9 0.9999238
9 0.9998313
9 0.9722116
9 0.9999571
9 0.99997795
9 0.9999473
9 0.9996526
9 0.999892
9 0.99982613
9 0.9994849
9 0.99982506
9 0.9999583
9 0.99976593
9 0.99980825
9 0.9998363
9 0.7089715
9 0.99827135
9 0.9999883
9 1.0
9 0.99999416
6 0.99547344
6 0.7444337
4 0.9990534
4 0.9265859
7 0.944015
7 0.8527844
7 0.9626648
7 0.9780107
7 0.9896248
7 0.95367354
7 0.9614806
7 0.9389301
7 0.96840805
7 0.93158996
7 0.96463484
7 0.97770417
7 0.9638525
7 0.9840635
7 0.89284545
10 0.5558883
10 0.99363905
10 0.9978593
10 0.9970361
10 0.9959579
10 0.99476886
10 0.9951687
10 0.9754068
10 0.9928185
10 0.9966444
10 0.9955674
10 0.9978102
10 0.9971643
10 

6 0.50486547
3 0.88672453
9 0.6992001
5 0.9982039
9 0.9999802
5 0.99823195
5 0.99941826
5 0.97932136
8 0.4551447
5 0.79159546
5 0.9954686
5 0.9978381
5 0.9998172
5 0.9983063
5 0.9997665
9 0.9997787
5 0.99971944
5 0.99525416
5 0.99923646
9 0.9972017
5 0.9742085
5 0.9905282
9 0.9479536
9 0.7814658
9 0.98739177
5 0.9940274
5 0.949232
5 0.99232715
5 0.99013585
5 0.9965767
5 0.83069056
5 0.9529902
5 0.9933622
5 0.9978908
3 0.97299045
5 0.9996414
5 0.98556596
9 0.95335805
9 0.46786088
5 0.86323124
5 0.99750084
5 0.9793297
5 0.9802699
5 0.980467
9 0.67504644
5 0.9867927
5 0.9986619
5 0.9470423
5 0.9985819
8 0.73303163
5 0.997054
5 0.49905634
5 0.99750704
5 0.94060963
5 0.5722009
5 0.99975795
3 0.95304614
5 0.62427235
9 0.79430753
3 0.81191003
5 0.9863391
5 0.9782598
5 0.98778653
3 0.9259508
5 0.9977266
5 0.62448287
5 0.98922825
5 0.77510446
5 0.9806997
5 0.8423024
1 0.68742335
5 0.9993918
5 0.9095373
9 0.58979017
5 0.99970955
5 0.88791007
5 0.93487793
5 0.9835812
5 0.9986457
3 0.95365167
5 0.

9 0.99994814
10 0.34545347
9 0.9998461
4 0.34589466
7 0.62043685
4 0.38522416
6 0.5406685
9 0.999897
4 0.9999486
4 0.9719016
4 0.9636074
4 0.8776703
9 0.96932507
9 0.82310164
10 0.9699516
4 0.9999641
9 0.5903329
9 0.9775854
4 0.5361119
4 0.95453715
4 0.99970776
4 0.99815327
5 0.9899685
9 0.95197886
9 0.49222508
9 0.6273468
4 0.71896166
4 0.9971988
9 0.9987564
9 0.9998498
4 0.9992951
9 0.50575656
7 0.56541914
9 0.9157648
9 0.9997049
9 0.999471
9 0.99999976
9 0.22075255
9 0.96092236
10 0.9473861
10 0.9506857
9 0.9390082
9 0.9985788
10 0.85183126
9 0.6947066
7 0.39892527
4 0.79922646
10 0.49984503
9 0.9999666
9 0.6170531
9 0.9992931
10 0.9839039
9 0.99975914
9 0.9397159
9 0.9999999
10 0.9951439


error: OpenCV(3.4.2) c:\miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<3,-1,-1>,struct cv::Set<0,5,-1>,2>::CvtHelper'
